# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [1]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Using TensorFlow backend.


Loaded dataset with 25000 training samples, 25000 test samples


In [2]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 14, 20, 9, 52, 18, 51, 12, 9, 5, 2, 13, 586, 106, 12, 1453, 190, 13, 244, 6, 826, 2, 2, 5, 198, 138, 13, 296, 12, 4, 118, 2, 7, 14, 20, 9, 2, 5, 2, 1079, 8, 85, 729, 102, 12, 127, 24, 2389, 94, 2194, 1437, 17, 257, 7, 4, 689, 935, 31, 3309, 2, 467, 1088, 234, 19, 4, 236, 467, 158, 52, 2, 2, 11, 4, 86, 2052, 189, 564, 2073, 39, 2, 2, 2, 8, 2, 154, 313, 2987, 2867, 29, 659, 60, 99, 76, 7, 12, 54, 29, 517, 8, 2, 31, 7, 27, 2615, 2, 114, 2, 729, 8, 2, 2, 52, 361, 7, 4, 213, 7, 650, 1009, 9, 4, 564, 397, 2985, 1815, 42, 17, 4, 277, 287, 27, 2, 322, 494, 41, 1461, 8, 297, 367, 19, 4, 2, 2, 8, 1278, 90, 1168, 2, 27, 2, 246, 80, 4, 2, 2, 2007, 8, 358, 4, 2, 7, 41, 2, 1347, 10, 10, 11, 4, 333, 2052, 107, 196, 58, 416, 369, 79, 295, 11, 4, 172, 2, 154, 313, 21, 566, 306, 8, 339, 2, 6, 1162, 189, 4559, 1890, 4, 2, 2, 7, 4, 196, 416, 119, 7, 68, 1940, 63, 69, 280, 2, 98, 39, 4, 3360, 4559, 2, 36, 850, 4, 255, 1000, 9, 6, 2, 2, 2, 2748, 42, 16, 59, 10, 10, 4, 840, 2052, 16, 4, 32

Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [3]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

--- Review (with words) ---
['the', 'as', 'on', 'it', 'very', 'but', 'when', 'that', 'it', 'to', 'and', 'was', 'attempt', 'character', 'that', 'zero', 'take', 'was', 'rather', 'is', 'viewing', 'and', 'and', 'to', 'series', 'such', 'was', 'plays', 'that', 'of', 'where', 'and', 'br', 'as', 'on', 'it', 'and', 'to', 'and', 'neither', 'in', 'because', 'sets', 'characters', 'that', 'end', 'his', 'contemporary', 'make', 'barbara', 'victim', 'movie', 'having', 'br', 'of', 'attention', 'america', 'by', 'rushed', 'and', '4', 'scott', 'since', 'film', 'of', 'performance', '4', "didn't", 'very', 'and', 'and', 'this', 'of', 'how', 'cameo', 'fact', 'violence', 'unexpected', 'or', 'and', 'and', 'and', 'in', 'and', 'work', 'everyone', 'oddly', 'elizabeth', 'all', 'talking', 'which', 'movies', 'get', 'br', 'that', 'no', 'all', 'gave', 'in', 'and', 'by', 'br', 'be', 'insight', 'and', 'little', 'and', 'sets', 'in', 'and', 'and', 'very', 'low', 'br', 'of', 'come', 'br', 'change', 'missing', 'it', 'of', 'v

In [4]:
maxlen = max([len(X_train[i]) for i in range(len(X_train))])
minlen = min([len(X_train[i]) for i in range(len(X_train))])

In [5]:
avglen = sum([len(X_train[i]) for i in range(len(X_train))])/len(X_train)

In [6]:
print(maxlen, minlen, avglen)

2494 11 238.71364


Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

maximum: 2494, minimum: 11


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [ ]:
import nltk
from nltk.corpus import stopwords

def clean_stopwords(lst):
    return [elem for elem in lst if elem not in stopwords.words('english')]

In [8]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test

X_train = sequence.pad_sequences(
    X_train, maxlen=max_words, dtype='int32', padding='pre', truncating='pre',
    value=0
)

X_test = sequence.pad_sequences(
    X_test, maxlen=max_words, dtype='int32', padding='pre', truncating='pre',
    value=0
)

'''
new_X_train = []
for i in range(len(X_train)):
    newrow = clean_stopwords(X_train[i])
    new_X_train.append(newrow[:500] if len(newrow) >= 500 else newrow+[0]*(500-len(newrow)))
X_train = new_X_train

new_X_test = []
for i in range(len(X_test)):
    newrow = clean_stopwords(X_test[i])
    new_X_test.append(newrow[:500] if len(newrow) >= 500 else newrow+[0]*(500-len(newrow)))

X_test = new_X_test
'''

'\nnew_X_train = []\nfor i in range(len(X_train)):\n    newrow = clean_stopwords(X_train[i])\n    new_X_train.append(newrow[:500] if len(newrow) >= 500 else newrow+[0]*(500-len(newrow)))\nX_train = new_X_train\n\nnew_X_test = []\nfor i in range(len(X_test)):\n    newrow = clean_stopwords(X_test[i])\n    new_X_test.append(newrow[:500] if len(newrow) >= 500 else newrow+[0]*(500-len(newrow)))\n\nX_test = new_X_test\n'

In [9]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [10]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Activation

# TODO: Design your model
model = Sequential()
model.add(Embedding(vocabulary_size, 50, input_length=max_words))
model.add(LSTM(20, activation='tanh', recurrent_dropout = 0.1))
model.add(Dense(2))
model.add(Activation('sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 20)                5680      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 42        
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 255,722
Trainable params: 255,722
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

In total, 255,722 parameters should be trained. I used one LSTM cell for input to get output

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [11]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [12]:
from keras.callbacks import EarlyStopping
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 32
num_epochs = 2

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)

# TODO: Train your model
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.1, callbacks = [callback])

Train on 22500 samples, validate on 2500 samples
Epoch 1/10
22500/22500 [==============================] - 315s 14ms/step - loss: 0.4926 - acc: 0.7682 - val_loss: 0.4399 - val_acc: 0.8096
Epoch 2/10
16992/22500 [=====================>........] - ETA: 1:15 - loss: 0.3438 - acc: 0.8614

KeyboardInterrupt: 

In [ ]:
# Save your model, so that you can quickly load it in future (and perhaps resume training)
import os

cache_dir = './'

model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join(cache_dir, model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

In [16]:
X_test = [X_test[i][:500] if len(X_test[i]) >= 500 else X_test[i]+[0]*(500-len(X_test[i])) for i in range(len(X_test))]

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [13]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=1)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.85792


In [11]:
print('I\'m Done!')

I'm Done!


#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

It is not fair to compair two models directly because they have huge difference in the number of parameters and training time. However under curreny my condition, Gradient-Boosted Decision Trees, both train & test accuracy was about 65%. However, in case of RNN shows 85% accuracy only in one epoch.

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?
    - to prevent overfitting, we can give dropout options on cells.

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)
    - we can make API to use or just simply save the model parameters to make pre-trained model.

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?
    - regression is for continuous value which has proportional attribute to label. classification is for descrete category which doesn't have relationship between label number and attribute of that label.

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!